In [1]:
import rioxarray as rioxr
import os
import numpy as np

from rasterio.crs import CRS
import geopandas as gpd

from shapely.geometry import box

In [2]:
# the two CRS that the NAIP scenes have
crs11 = CRS.from_epsg(26911)
crs10 = CRS.from_epsg(26910)

In [9]:
year = 2012

In [10]:
# directory where the NAIP classifications are
dirp = os.path.join(os.getcwd(),
                   'S_clip_preds',
                    'S_clip_preds_' + str(year))

# list of files in that directory
tifs = os.listdir(dirp)

# directories for scenes with crs= epsg(26911) and rs= epsg(26910)
fp_11 = os.path.join(dirp, 'crs_26911')
fp_10 = os.path.join(dirp, 'crs_26910')

# make directories
os.mkdir(fp_10)
os.mkdir(fp_11)

# **********************************************************
# move files to directories according to their CRS
for tif in tifs:
    fp = os.path.join(dirp, tif)
    
    if rioxr.open_rasterio(fp).rio.crs == crs11:
        newfp = os.path.join(fp_11, tif)
    if rioxr.open_rasterio(fp).rio.crs == crs10:
        newfp = os.path.join(fp_10, tif)
        
    os.rename(fp, newfp)
    
# **********************************************************
# SPLIT crs26910 into south and west coast

# list of files in crs10 folder
tifs = os.listdir(fp_10)

# directories for scenes in west and south coasts
fp_w = os.path.join(fp_10, 'west_coast')
fp_s = os.path.join(fp_10, 'south_coast')

# make directories
os.mkdir(fp_w)
os.mkdir(fp_s)

# open bounding box of SB county west coast and match crs
wcoast = gpd.read_file(os.path.join(os.getcwd(),'sb_west_coast','sb_west_coast.shp')).to_crs(crs10)

# move files to directories according to their coast location
for tif in tifs:
    
    fp = os.path.join(fp_10, tif)
    x = rioxr.open_rasterio(fp)
    bbox = box(*x.rio.bounds())
    
    if bbox.intersects(wcoast.geometry[0]):
        newfp = os.path.join(fp_w, tif)
    else:
        newfp = os.path.join(fp_s, tif)
        
    os.rename(fp, newfp)